In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os

from tensorflow.keras import layers, models, optimizers, datasets
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Activation,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization
from tensorflow.python.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.optimizers import Adam



train_dirs = ['/root/images/ED1', '/root/images/ED2_GaussianNoisy',
             '/root/images/ED3_NotBalancedGan', '/root/images/ED4_BalancedGan',
             '/root/images/ED5_ColorJittering', '/root/images/ED6_MirrorHorizontally',
             '/root/images/ED7_GaussianBlur', '/root/images/ED8_MixUp']  

2023-06-07 19:17:31.156220: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# 绘制混淆矩阵
def plot_confusion_matrix(model, test_generator, dataset_name):
    # 评估模型在测试集上的准确率
    test_loss, test_acc = model.evaluate(test_generator, verbose=2)
    print("Test Accuracy: {:.2f}%".format(test_acc * 100))

    # 预测测试集的类别
    y_true = test_generator.classes
    y_pred = model.predict(test_generator)
    y_pred = np.argmax(y_pred, axis=1)

    # 计算混淆矩阵
    cm = confusion_matrix(y_true, y_pred)

    # 绘制混淆矩阵图像
    class_names = list(test_generator.class_indices.keys())
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    # plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()

    # 在每个格子中添加对应的数字
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    save_dir = 'Alexnet_Result_img'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 保存混淆矩阵图像
    plt.savefig(os.path.join(save_dir, dataset_name + 'confusion_matrix.png'), dpi=600)

In [3]:
# 定义AlexNet模型
def AlexNet(input_shape=(128,128,3),output_shape=3):
    model = Sequential()
    # 使用步长为4x4，大小为11的卷积核对图像进行卷积
    # 所建模型后输出为48特征层
    model.add(
        Conv2D(
            filters=48, 
            kernel_size=(11,11),
            strides=(4,4),
            padding='valid',
            input_shape=input_shape,
            activation='relu'
        )
    )
    
    model.add(BatchNormalization())
    # 使用步长为2的最大池化层进行池化
    model.add(
        MaxPooling2D(
            pool_size=(3,3), 
            strides=(2,2), 
            padding='valid'
        )
    )
    # 使用步长为1x1，大小为5的卷积核对图像进行卷积
    # 所建模型后输出为128特征层
    model.add(
        Conv2D(
            filters=128, 
            kernel_size=(5,5), 
            strides=(1,1), 
            padding='same',
            activation='relu'
        )
    )
    
    model.add(BatchNormalization())
    # 使用步长为2的最大池化层进行池化
    model.add(
        MaxPooling2D(
            pool_size=(3,3),
            strides=(2,2),
            padding='valid'
        )
    )
    # 使用步长为1x1，大小为3的卷积核对图像进行卷积
    # 所建模型后输出为192特征层
    model.add(
        Conv2D(
            filters=192, 
            kernel_size=(3,3),
            strides=(1,1), 
            padding='same', 
            activation='relu'
        )
    ) 
    # 使用步长为1x1，大小为3的卷积核对图像进行卷积
    # 所建模型后输出为192特征层
    model.add(
        Conv2D(
            filters=192, 
            kernel_size=(3,3), 
            strides=(1,1), 
            padding='same', 
            activation='relu'
        )
    )
    # 使用步长为1x1，大小为3的卷积核对图像进行卷积
    # 所建模型后输出为128特征层
    model.add(
        Conv2D(
            filters=128, 
            kernel_size=(3,3), 
            strides=(1,1), 
            padding='same', 
            activation='relu'
        )
    )
    # 使用步长为2的最大池化层进行池化；
    model.add(
        MaxPooling2D(
            pool_size=(3,3), 
            strides=(2,2), 
            padding='valid'
        )
    )
    # 两个全连接层，最后输出为1000类,这里改为3类
    # 缩减为1024
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    # model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    # model.add(Dropout(0.25))
    model.add(Dense(output_shape, activation='softmax'))
    return model

# 创建模型实例
model = AlexNet()

# 学习率下降的方式，acc三次不下降就下降学习率继续训练
reduce_lr = ReduceLROnPlateau(
                        monitor='acc', 
                        factor=0.5, 
                        patience=3, 
                        verbose=1
                    )
# 是否需要早停，当val_loss一直不下降的时候意味着模型基本训练完毕，可以停止
early_stopping = EarlyStopping(
                        monitor='val_loss', 
                        min_delta=0, 
                        patience=10, 
                        verbose=1
)
# 编译模型。交叉熵
model.compile(loss = 'categorical_crossentropy',
        optimizer = Adam(lr=1e-3),
        metrics = ['accuracy'])

print(model.summary())

2023-06-07 19:17:33.240970: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-07 19:17:33.349117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:52:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2023-06-07 19:17:33.349169: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-07 19:17:33.354120: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-07 19:17:33.354243: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2023-06-07 19:17:33.355618: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcu

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 48)        17472     
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 48)        192       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 48)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       153728    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 192)         2

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
for i, train_dir_NP in enumerate(train_dirs):
    
    # 当前数据集名字
    dataset_name = train_dir_NP.split('/')[-1]
    
    # 定义数据生成器
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    
    train_dir = train_dir_NP
    val_dir = train_dir_NP
    test_dir = '/root/images/test'

    # 加载训练集和验证集
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        val_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    # 加载测试集
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    # 训练模型并绘制准确率图像
    history = model.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator
    )
    
    
        
    # 保存模型参数
    save_dir = '/root/Alexnet_Result_img'
    model_dir = os.path.join(save_dir, dataset_name)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model.save_weights(os.path.join(model_dir, 'model_weights.h5'))

    
    # 创建保存图像的文件夹
    save_dir = 'Alexnet_Result_img'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    # 提取训练集和验证集的准确率
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    # 提取训练集和验证集的损失
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    # 绘制准确率图像
    plt.figure()
    epochs = range(1, len(train_acc) + 1)
    plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'orange', label='Validation Accuracy')
    # plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(os.path.join(save_dir, dataset_name+'accuracy_plot.png'), dpi=1200)

    
    # 绘制损失图像
    plt.figure()
    plt.plot(epochs, train_loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'orange', label='Validation Loss')
    # plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(save_dir, dataset_name+'loss_plot.png'), dpi=1200)
    
    # 绘制混淆矩阵
    plot_confusion_matrix(model, test_generator, dataset_name)
    
    print("--------",dataset_name, "-------- Over")

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Found 750 images belonging to 3 classes.


2023-06-07 19:17:35.459008: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-06-07 19:17:35.487745: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2600000000 Hz


Epoch 1/50


2023-06-07 19:17:36.236126: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-06-07 19:17:37.165823: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2023-06-07 19:17:38.269402: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-07 19:17:39.106286: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2023-06-07 19:17:39.215723: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


75/75 [==============================] - 8s 48ms/step - loss: 0.9465 - accuracy: 0.6004 - val_loss: 1.4611 - val_accuracy: 0.3783
Epoch 2/50
75/75 [==============================] - 3s 42ms/step - loss: 0.7432 - accuracy: 0.7017 - val_loss: 0.8300 - val_accuracy: 0.6583
Epoch 3/50
75/75 [==============================] - 3s 37ms/step - loss: 0.6856 - accuracy: 0.7254 - val_loss: 1.1528 - val_accuracy: 0.4817
Epoch 4/50
75/75 [==============================] - 3s 35ms/step - loss: 0.6306 - accuracy: 0.7500 - val_loss: 0.7334 - val_accuracy: 0.6750
Epoch 5/50
75/75 [==============================] - 3s 35ms/step - loss: 0.5919 - accuracy: 0.7625 - val_loss: 0.6363 - val_accuracy: 0.7417
Epoch 6/50
75/75 [==============================] - 3s 34ms/step - loss: 0.5693 - accuracy: 0.7696 - val_loss: 1.0302 - val_accuracy: 0.6017
Epoch 7/50
75/75 [==============================] - 3s 36ms/step - loss: 0.4959 - accuracy: 0.8004 - val_loss: 0.5920 - val_accuracy: 0.7883
Epoch 8/50
75/75 [======